Import necessary packages

In [ ]:
!pip install scikit-learn==1.3.2 --no-cache-dir

In [ ]:
# import the necessary packages
import tensorflow as tf
import numpy as np
import pandas as pd
from pandas import read_csv
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score
from imblearn.metrics import specificity_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import matplotlib.pyplot as plt
import sklearn
from itertools import product
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, anneal
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.ensemble import ExtraTreesClassifier

Declare the necessary variables


In [ ]:
# declare the necessary variables
train_data_file = "train_data.csv"
test_data_file = "test_data.csv"
all_data_file = "all_data.csv"
X_data_file = "X_data.csv"
Y_data_file = "Y_data.csv"
TARGET_LABELS = [0.0,1.0]

Prepare the data


In [ ]:
# prepare data
drive.mount('/content/drive')

# replace with your Google Drive link in np.load()
cambridgeAsymptomaticdatasetX = np.load('/content/drive/MyDrive/data/Cambridge/Task 1/cough_X_features_np.npy')
cambridgeAsymptomaticdatasetY = np.load('/content/drive/MyDrive/data/Cambridge/Task 1/cough_y_features_np.npy')

cambridgeSymptomaticdatasetX = np.load('/content/drive/MyDrive/data/Cambridge/Task 2/cough_X_features_np.npy')
cambridgeSymptomaticdatasetY = np.load('/content/drive/MyDrive/data/Cambridge/Task 2/cough_y_features_np.npy')

coswaradatasetX = np.load('/content/drive/MyDrive/data/Coswara/cough_X_features_np.npy')
coswaradatasetY = np.load('/content/drive/MyDrive/data/Coswara/cough_y_features_np.npy')

coughviddatasetX = np.load('/content/drive/MyDrive/data/COUGHVID/cough_X_features_np.npy')
coughviddatasetY = np.load('/content/drive/MyDrive/data/COUGHVID/cough_y_features_np.npy')

virufydatasetX = np.load('/content/drive/MyDrive/data/Virufy/cough_X_features_np.npy')
virufydatasetY = np.load('/content/drive/MyDrive/data/Virufy/cough_y_features_np.npy')

virufyNoCoCoDadatasetX = np.load('/content/drive/MyDrive/data/NOCOCODA and Virufy/cough_X_features_np.npy')
virufyNoCoCoDadatasetY = np.load('/content/drive/MyDrive/data/NOCOCODA and Virufy/cough_y_features_np.npy')

ComPareX = np.load('/content/drive/MyDrive/data/ComPare/cough_X_features_np.npy')
ComPareY = np.load('/content/drive/MyDrive/data/ComPare/cough_y_features_np.npy')


# cambridgeAsymptomaticdataset,cambridgeSymptomaticdataset,coswaradataset,coughviddataset,virufydataset,virufyNoCoCoDadataset
datasetX = virufydatasetX # Replace dataset name + 'X'
datasetY = virufydatasetY # Replace dataset name + 'Y'

train_index = []
test_index = []

# split train and test data
for i in range(10):
   # replace dataset name (cambridgeAsymptomaticdataset,cambridgeSymptomaticdataset,coswaradataset,coughviddataset,virufydataset,virufyNoCoCoDadataset)
   train_index_url = '/content/drive/MyDrive/covid-19 cough sound dataset/virufydataset/train/'+str(i)+'.csv'
   test_index_url = '/content/drive/MyDrive/covid-19 cough sound dataset/virufydataset/test/'+str(i)+'.csv'

   df = pd.read_csv(train_index_url)
   label=df['train_index'].tolist()
   train_array=np.array(label)
   train_index.append(train_array)

   df = pd.read_csv(test_index_url)
   label=df['test_index'].tolist()
   test_array=np.array(label)
   test_index.append(test_array)


print(f"cambridgeAsymptomaticdatasetX shape : {cambridgeAsymptomaticdatasetX.shape}")
print(f"cambridgeAsymptomaticdatasetY shape : {cambridgeAsymptomaticdatasetY.shape}")
print('')

print(f"cambridgeSymptomaticdatasetX shape : {cambridgeSymptomaticdatasetX.shape}")
print(f"cambridgeSymptomaticdatasetY shape : {cambridgeSymptomaticdatasetY.shape}")
print('')


print(f"coswaradatasetX shape : {coswaradatasetX.shape}")
print(f"coswaradatasetY shape : {coswaradatasetY.shape}")


print(f"coughviddatasetX shape : {coughviddatasetX.shape}")
print(f"coughviddatasetY shape : {coughviddatasetY.shape}")


print(f"virufydatasetX shape : {virufydatasetX.shape}")
print(f"virufydatasetY shape : {virufydatasetY.shape}")


print(f"virufyNoCoCoDadatasetX shape : {virufyNoCoCoDadatasetX.shape}")
print(f"virufyNoCoCoDadatasetY shape : {virufyNoCoCoDadatasetY.shape}")


print(f"ComPareX shape : {ComPareX.shape}")
print(f"ComPareY shape : {ComPareY.shape}")


X_data_ls = []
Y_data_ls = []

rows,columns = datasetX.shape

C=0
N=0

for i in range(rows):
    col = []
    arr = datasetX[i]
    for j in range(1,len(arr)+1,1):
       col.append(float(arr[j-1]))
    X_data_ls.append(col)
    if(datasetY[i]=='C'):
       Y_data_ls.append(0)
       C+=1
    else:
       Y_data_ls.append(1)
       N+=1

X_data = np.array(X_data_ls)
Y_data = np.array(Y_data_ls)

print(f"X_data: {X_data.shape}")
print(f"Y_data: {Y_data.shape}")

print(f"C: {C}")
print(f"N: {N}")

RFECV+Extra-trees

In [ ]:
min_features_to_select = 1  # Minimum number of features to consider

# feature selection
#def select_features(X_train, y_train, X_test):
def select_features(X_train,y_train,X_data):
  # Create the RFE object and compute a cross-validated score.
  #fs_model = RandomForestClassifier(n_estimators = 800,random_state = 0)
  fs_model = ExtraTreesClassifier(n_estimators=50,random_state = 0)
  #fs_model = LinearSVC(C=0.01, penalty="l1", dual=False)
  rfecv = RFECV(estimator=fs_model, step=1, cv=StratifiedKFold(10),scoring='roc_auc',min_features_to_select=min_features_to_select)

  print(f"shape of X_data before transform: {X_data.shape}")

  rfecv.fit(X_train, y_train)

  # transform train input data
  X_data_fs = rfecv.transform(X_data)
  print(f"shape of X_data_fs after transform: {X_data_fs.shape}")

  return X_data_fs, rfecv

X_train, X_test, y_train, y_test = train_test_split(X_data,Y_data,test_size=0.20, random_state=42)
X_data_fs, rfecv = select_features(X_train,y_train,X_data)

In [ ]:
rows,columns = X_data_fs.shape

TARGET_FEATURE_NAME = str(columns+1)
#Derive the necessary variables
#No. of classes, assuming last column is dependent variable
num_classes = len(TARGET_LABELS)

#print([x for x in range(5)])
#List of all the columns
CSV_HEADER = []
for i in range(columns+1):
    CSV_HEADER.append(str(i+1))

# A list of the numerical feature names.
#List of only independent variables
FEATURE_NAMES = []

for i in range(columns):
    FEATURE_NAMES.append(str(i+1))

COLUMN_DEFAULTS = [
    [0.0] if feature_name in CSV_HEADER  else ["NA"]
    for feature_name in CSV_HEADER
]

Bayesian optimization

In [ ]:
print(f"X_data: {X_data.shape}")
print(f"Y_data: {Y_data.shape}")

"""
# You can try hyperparameter tuning with Bayesian Optimization to gain hands-on experience.

# installing library for Bayesian optimization
!pip install hyperopt

X = X_data
y = Y_data


##############################################################
# Bayesian hyperparameter optimization

RF = RandomForestClassifier()
ET = ExtraTreesClassifier()

batch_sz_list = [8,16,32,64,128,256]

# Defining the hyper parameter space as a dictionary
parameter_space = { 'num_trees': hp.quniform('num_trees',5,50,1),
                    'depth': hp.quniform('depth',3,50,1),
                    'used_features_rate': hp.quniform('used_features_rate',0.5,1,.1),
                    'batch_size': hp.choice('batch_size',batch_sz_list),
                    'num_epochs': hp.quniform('num_epochs', 10,50,1)
                  }

# Defining a cost function which the Bayesian algorithm will optimize
def objective(parameter_space):

    # The accuracy parameter is the average accuracy obtained by cross validation of the data
    # See different scoring methods by using sklearn.metrics.SCORERS.keys()
    Error = cross_val_score(ET, X, y, cv = 10, scoring='accuracy').mean()

    # We return the loss which will be minimized by the fmin() function
    return {'loss': -Error, 'status': STATUS_OK }

import warnings
warnings.filterwarnings('ignore')

# Finding out which set of hyperparameters give highest accuracy
trials = Trials()
best_params = fmin(fn= objective,
            space= parameter_space,
            #algo= tpe.suggest,
            algo=anneal.suggest,  # the logic which chooses next parameter to try
            max_evals = 100,
            trials= trials)
#The best hyperparameters are returned by the function ‘fmin()’.
#We have stored the results in the ‘best_params’ variable.
print('The best parameters are:', best_params)

learning_rate = 0.01
num_trees = int(best_params['num_trees'])
depth = int(best_params['depth'])
used_features_rate = round(best_params['used_features_rate'],1)
batch_size = int(batch_sz_list[best_params['batch_size']])
num_epochs = int(best_params['num_epochs'])

print(f"num_trees : {num_trees} depth : {depth}  used_features_rate : {used_features_rate}")
print(f"learning_rate : {learning_rate}  batch_size : {batch_size}  num_epochs : {num_epochs}")
"""

"""
# To save time on Bayesian Optimization, use the hyperparameters below, which have already been calculated for optimal performance.

#for strategy1 && strategy2 && strategy3 (Without hyperparameter tuning via Bayesian Optimization)
num_trees = 10
depth = 10
used_features_rate = 1.0
learning_rate = 0.01
batch_size = 256
num_epochs = 10


#for strategy4 && strategy5 (Bayesian Optimization)
#cambridge_asymptomatic_eT
num_trees = 18
depth = 8
used_features_rate = 0.8
learning_rate = 0.01
batch_size = 32
num_epochs = 17

#cambridge_symptomatic_eT
num_trees = 25
depth = 9
used_features_rate = 0.8
learning_rate = 0.01
batch_size = 8
num_epochs = 13

#coswara_eT
num_trees = 25
depth = 11
used_features_rate = 0.6
learning_rate = 0.01
batch_size = 16
num_epochs = 14

#coughvid_eT
num_trees = 17
depth = 6
used_features_rate = 1.0
learning_rate = 0.01
batch_size = 32
num_epochs = 19

#virufy_eT
num_trees = 16
depth = 16
used_features_rate = 0.8
learning_rate = 0.01
batch_size = 8
num_epochs = 22

#Virufy + NoCoCoDa_eT
num_trees = 29
depth = 5
used_features_rate = 0.6
learning_rate = 0.01
batch_size = 32
num_epochs = 34
"""

num_trees = 16
depth = 16
used_features_rate = 0.8
learning_rate = 0.01
batch_size = 8
num_epochs = 22


In [ ]:
# import the data from csv with all the relevant columns
def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128, num_epochs=1):
    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        column_defaults=COLUMN_DEFAULTS,
        label_name=TARGET_FEATURE_NAME,
        num_epochs=num_epochs,
        #num_epochs=1,
        header=False,
        na_value="?",
        shuffle=shuffle,
    ).map(lambda features, target: (features, target))
    return dataset.cache()

# create one input layer for each feature
def create_model_inputs(FEATURE_NAMES):
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(), dtype=tf.float32)
    return inputs

# If there is any categorical feature, it can be encoded
# Since we had only numerical feature, we will skip this step
# We will also exdpand the dimension of the feature inputs
def encode_inputs(inputs):
    encoded_features = []
    for feature_name in inputs:
        encoded_feature = inputs[feature_name]
        if inputs[feature_name].shape[-1] is None:
            encoded_feature = tf.expand_dims(encoded_feature, -1)

        encoded_features.append(encoded_feature)

    encoded_features = layers.concatenate(encoded_features)
    return encoded_features

In [ ]:
class NeuralDecisionTree(keras.Model):
    def __init__(self, depth, num_features, used_features_rate, num_classes):
        super(NeuralDecisionTree, self).__init__()
        self.depth = depth   # Pre-defined depth
        self.num_leaves = 2 ** depth  # No of leaves in the tree
        self.num_classes = num_classes  # No of classes in the dependent variable

        # Create a mask for the randomly selected features.
        # Number of features to be selected for each tree
        num_used_features = int(num_features * used_features_rate)
        # Select "num_used_features" features from the total features
        one_hot = np.eye(num_features)
        """
        sampled_feature_indicies = np.random.choice(
            np.arange(num_features), num_used_features, replace=False
        )
        """
        try:
             sampled_feature_indicies = np.random.choice(np.arange(num_features), num_used_features, replace=False)
        except ValueError:
             sampled_feature_indicies = np.random.choice(np.arange(num_features), num_used_features, replace=True)

        self.used_features_mask = one_hot[sampled_feature_indicies]

        # Initialize the weights of the classes in leaves.
        self.pi = tf.Variable(
            initial_value=tf.random_normal_initializer()(
                shape=[self.num_leaves, self.num_classes]
            ),
            dtype="float32",
            trainable=True,
        )

        # Initialize the stochastic routing layer.
        self.decision_fn = layers.Dense(
            units=self.num_leaves, activation="sigmoid", name="decision"
        )

    def call(self, features):
        batch_size = tf.shape(features)[0]

        # Apply the feature mask to the input features.
        features = tf.matmul(
            features, self.used_features_mask, transpose_b=True
        )
        # Compute the routing probabilities.
        decisions = tf.expand_dims(
            self.decision_fn(features), axis=2
        )
        # Concatenate the routing probabilities with their complements.
        decisions = layers.concatenate(
            [decisions, 1 - decisions], axis=2
        )
        # Initiate mu, the probablity of a sample reaching a leaf node
        mu = tf.ones([batch_size, 1, 1])

        begin_idx = 1
        end_idx = 2
        # Traverse the tree in breadth-first order.
        # Update probabilities in each level and node.
        # Calculate total final output probability
        for level in range(self.depth):
            mu = tf.reshape(mu, [batch_size, -1, 1])  # [batch_size, 2 ** level, 1]
            mu = tf.tile(mu, (1, 1, 2))  # [batch_size, 2 ** level, 2]
            level_decisions = decisions[
                :, begin_idx:end_idx, :
            ]  # [batch_size, 2 ** level, 2]
            mu = mu * level_decisions  # [batch_size, 2**level, 2]
            begin_idx = end_idx
            end_idx = begin_idx + 2 ** (level + 1)
        mu = tf.reshape(mu, [batch_size, self.num_leaves])  # [batch_size, num_leaves]
        probabilities = keras.activations.softmax(self.pi)  # [num_leaves, num_classes]
        outputs = tf.matmul(mu, probabilities)  # [batch_size, num_classes]
        return outputs

In [ ]:
class NeuralDecisionForest(keras.Model):
    def __init__(self, num_trees, depth, num_features, used_features_rate, num_classes):
        super(NeuralDecisionForest, self).__init__()
        self.ensemble = []
        # Initialize the ensemble by adding NeuralDecisionTree instances.
        # Each tree will have its own randomly selected input features to use.
        for _ in range(num_trees):
            self.ensemble.append(
                NeuralDecisionTree(depth, num_features, used_features_rate, num_classes
                )
            )

    def call(self, inputs):
        # Initialize the outputs: a [batch_size, num_classes] matrix of zeros.
        batch_size = tf.shape(inputs)[0]
        outputs = tf.zeros([batch_size, num_classes])

        # Aggregate the outputs of trees in the ensemble.
        for tree in self.ensemble:
            outputs += tree(inputs)
        # Divide the outputs by the ensemble size to get the average.
        outputs /= len(self.ensemble)
        return outputs

In [ ]:
def create_tree_model():
    inputs = create_model_inputs(FEATURE_NAMES)
    features = encode_inputs(inputs)
    features = layers.BatchNormalization()(features)
    num_features = features.shape[1]
    tree = NeuralDecisionTree(depth, num_features, used_features_rate, num_classes)

    outputs = tree(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# create the forest model by taking input, output and the model structure
def create_forest_model():
    inputs = create_model_inputs(FEATURE_NAMES)
    features = encode_inputs(inputs)
    features = layers.BatchNormalization()(features)
    num_features = features.shape[1]
    forest_model = NeuralDecisionForest(
        num_trees, depth, num_features, used_features_rate, num_classes
    )

    outputs = forest_model(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

SMOTE and Threshold moving (ROC-AUC score)



In [ ]:
from numpy import arange
from numpy import argmax
from sklearn.metrics import f1_score
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE

"""
On a binary classification problem with class labels 0 and 1, normalized predicted probabilities and a threshold of 0.5, then values less than the
threshold of 0.5 are assigned to class 0 and values greater than or equal to 0.5 are assigned to class 1.

Prediction < 0.5 = Class 0
Prediction >= 0.5 = Class 1
"""

# define thresholds
thresholds = arange(0, 1, 0.001)

# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

# now compile the model, train it on train sample and predict it for test sample.
def run_experiment(model):

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )


    lst_roc_auc = []
    lst_precision = []
    lst_recall = []
    lst_f1 = []
    lst_acc = []
    lst_speci = []

    skf = StratifiedKFold(n_splits=10, shuffle=True)

    for i in range(10):
        X_train, X_test = X_data_fs[train_index[i]], X_data_fs[test_index[i]]
        y_train, y_test = Y_data[train_index[i]], Y_data[test_index[i]]

        print('Before Oversampling')
        print(f'X_train shape: {X_train.shape}')
        print(f'y_train shape: {y_train.shape}')
        counter=Counter(y_train)
        print(counter)
        oversample = SVMSMOTE()
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        print('After Oversampling')
        print(f'X_train shape: {X_train.shape}')
        print(f'y_train shape: {y_train.shape}')
        counter = Counter(y_train)
        print(counter)

        all_data_train_fold = []
        rows,columns = X_train.shape

        for i in range(rows):
           col2 = []
           arr = X_train[i]
           for j in range(1,len(arr)+1,1):
              col2.append(float(arr[j-1]))
           col2.append(y_train[i])
           all_data_train_fold.append(col2);

        print(f'all_data_train_fold shape: {np.array(all_data_train_fold).shape}')

        all_data_test_fold = []
        rows,columns = X_test.shape

        for i in range(rows):
           col2 = []
           arr = X_test[i]
           for j in range(1,len(arr)+1,1):
              col2.append(float(arr[j-1]))
           col2.append(y_test[i])
           all_data_test_fold.append(col2);

        print(f'all_data_test_fold shape: {np.array(all_data_test_fold).shape}')

        # convert array into dataframe
        DF = pd.DataFrame(all_data_train_fold)
        # save the dataframe as a csv file
        DF.to_csv(train_data_file, index=False, header=False)
        train_data = get_dataset_from_csv(
           train_data_file, batch_size=batch_size, num_epochs=num_epochs
        )

        model.fit(train_data)

        # convert array into dataframe
        DF1 = pd.DataFrame(all_data_test_fold)
        # save the dataframe as a csv file
        DF1.to_csv(test_data_file, index=False, header=False)
        test_data = get_dataset_from_csv(
           test_data_file, batch_size=batch_size
        )
        #_, accuracy = model.evaluate(test_data)
        #lst_accu_stratified.append(round(accuracy, 2))

        #print(f"y_test: {y_test}")
        true_categories = tf.concat([y for x, y in test_data], axis=0)
        print(f"true_categories: {true_categories}")

        # keep probabilities for the positive outcome only
        Y_pred = model.predict(test_data)
        #print(Y_pred)
        probs = Y_pred[:, 1]
        #print(f"probs: {probs}")

        # evaluate each threshold
        #scores = [precision_score(true_categories, to_labels(probs, t)) for t in thresholds]
        #scores = [f1_score(true_categories, to_labels(probs, t)) for t in thresholds]
        scores = [roc_auc_score(true_categories, to_labels(probs, t)) for t in thresholds]
        # get best threshold
        ix = argmax(scores)
        #print(f"Optimal threshold score: {ix}")
        #print(f"Thresholds[ix]: {thresholds[ix]}")

        #Y_pred = model.predict(test_data)
        #probs = Y_pred[:, 1]
        #print(probs)
        #print(f"probsLast: {probs}")
        #print(f"to_labels(probs, thresholds[ix]): {to_labels(probs[len(probs)-1], thresholds[ix])}")
        print(f"TM: {thresholds[ix]}")
        print(f"To Labels(probs, thresholds[ix]): {to_labels(probs, thresholds[ix])}")

        conf_matrix = confusion_matrix(true_categories, to_labels(probs, thresholds[ix]))
        print(conf_matrix)

        tp, fn, fp, tn = conf_matrix.ravel()
        print(f"True positive: {tp}")
        print(f"True negative: {tn}")
        print(f"False positive: {fp}")
        print(f"False negative: {fn}")

        lst_roc_auc.append(roc_auc_score(true_categories,to_labels(probs, thresholds[ix])));

        if(tp+fp == 0):
           lst_precision.append(0)
        else:
           lst_precision.append(tp/(tp+fp));

        if(tp+fn == 0):
           lst_recall.append(0)
        else:
           lst_recall.append(tp/(tp+fn));

        print(f"roc_auc_score : {roc_auc_score(true_categories,to_labels(probs, thresholds[ix]))}")
        #print(f"precision : {precision_score(true_categories,to_labels(probs, thresholds[ix]))}")
        #print(f"recall_score : {recall_score(true_categories,to_labels(probs, thresholds[ix]))}")

        print('')

    print(f"mean_of_roc_auc_score : {mean(lst_roc_auc)}")
    print(f"mean_of_precision : {mean(lst_precision)}")
    print(f"mean_of_recall : {mean(lst_recall)}")

In [ ]:
class ExpandDimsLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.expand_dims(inputs, -1)

# If there is any categorical feature, it can be encoded
# Since we had only numerical feature, we will skip this step
# We will also exdpand the dimension of the feature inputs
def encode_inputs(inputs):
    encoded_features = []
    for feature_name in inputs:
        encoded_feature = inputs[feature_name]
        if inputs[feature_name].shape[-1] is None:
            encoded_feature = ExpandDimsLayer()(encoded_feature)

        encoded_features.append(encoded_feature)

    encoded_features = layers.concatenate(encoded_features)
    return encoded_features

DNDT model creation and run

In [ ]:
tree_model = create_tree_model()
run_experiment(tree_model)

DNDF model creation and run

In [ ]:
forest_model = create_forest_model()
run_experiment(forest_model)